<a href="https://colab.research.google.com/github/Manoj048/Car-price-Regression/blob/main/02_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification



In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import required modules

In [33]:
import numpy as np
import pandas as pd
import os
import re

### Data pre-processing

In [34]:
data=pd.read_csv("/content/drive/MyDrive/NLP day_1/SMSSpamCollection.csv",delimiter="\t") #Reading data
data.columns=['Type','Text'] #Changing Column names 
type(data) #Type is a data frame
data.head(5) #Displays first five records

,Type,Text
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [35]:
data.isnull().sum()

Type    0
Text    0
dtype: int64

In [36]:
Text_data = data['Text']
type(data)


pandas.core.frame.DataFrame

In [37]:
['OK lar...joking..','Free entry in 2...',]

['OK lar...joking..', 'Free entry in 2...']

In [38]:
Text_data1 = Text_data.tolist()
#Some text preprocessing- Removing extra spaces and numbers
for i in range(len(Text_data1)):   #len will give number of messages in the text
    Text_data1[i]=re.sub(r'\s+', ' ', Text_data1[i]) #Removing more than one white spaces in the sentence
    Text_data1[i]=re.sub('[\d]', '',Text_data1[i]) #Removing numbers


In [39]:
#Text_data1 = Text_data.tolist()
Text_data1[8]

'Had your mobile  months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on '

In [40]:
Text_data1[0:10]

['Ok lar... Joking wif u oni...',
 "Free entry in  a wkly comp to win FA Cup final tkts st May . Text FA to  to receive entry question(std txt rate)T&C's apply over's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though",
 "FreeMsg Hey there darling it's been  week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £. to rcv",
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press * to copy your friends Callertune",
 'WINNER!! As a valued network customer you have been selected to receivea £ prize reward! To claim call . Claim code KL. Valid  hours only.',
 'Had your mobile  months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on ',
 "I'm gonna be home soon and

### Split the data into train and test

In [43]:
from sklearn.model_selection import train_test_split 
X = data['Text']
y = data['Type']
X_train, X_test, y_train, y_test=train_test_split(Text_data1,data['Type'],test_size=0.3,random_state=1234)

In [44]:
#Ensure similar distribution of target classes in both train and test
y_train.value_counts()

ham     3378
spam     521
Name: Type, dtype: int64

In [45]:
y_test.value_counts()

ham     1446
spam     226
Name: Type, dtype: int64

### Construct tf-idf matrix

In [48]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
count_vect=CountVectorizer()
train_counts=count_vect.fit_transform(X_train)
train_counts.shape
#count=CountVectorizer(strip_accents='unicode',ngram_range=(1,1),stop_words='english')
#tfreq=count.fit_transform(Text_data1)
#Dense_mat=tfreq.todense()
#A=pd.DataFrame(Dense_mat,)columns=count.get_feature_names())

(3899, 6464)

In [49]:
#Lets construct TF-idf matrix based on the train documents
tfidf_transformer = TfidfVectorizer(ngram_range=(1,1),stop_words='english')
X_train_tfidf = tfidf_transformer.fit_transform(X_train)
X_train_tfidf.shape

(3899, 6206)

In [50]:
#??tfidf_transformer.fit #"""Learn vocabulary and idf from training set.
#??tfidf_transformer.transform # """Transform documents to document-term matrix.
#Uses the vocabulary and document frequencies (df) learned by fit (or fit_transform)

In [51]:
# Get the tfidf matrix for test documents
X_test_tfidf = tfidf_transformer.transform(X_test)
X_test_tfidf.shape

(1672, 6206)

### Implementing Naive Bayes Classification model

In [54]:
#Pipeline

In [56]:
from sklearn.svm import LinearSVC
clf=LinearSVC()
clf.fit(X_train_tfidf,y_train)



LinearSVC()

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
text_clf=Pipeline([('tfidf',TfidfVectorizer()),
                    ('clf',LinearSVC()),
 ])
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [62]:
predictions=text_clf.predict(X_test)

In [63]:
predictions

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [65]:
 #MOVIES

In [52]:
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB().fit(X_train_tfidf, y_train) #nb_clf is the classifier
pred_train = nb_clf.predict(X_train_tfidf) #get predictions on train data

from sklearn.metrics import confusion_matrix
prediction=confusion_matrix(y_train, pred_train) #get confusion matrix for train predictions


##### Metrics on train data

In [ ]:
#Metrics
print("Train_Confusion Matrix:\n",prediction,"\n")

from sklearn.metrics import recall_score, precision_score, accuracy_score

acc = accuracy_score(y_train, pred_train)
rec = recall_score(y_train, pred_train, pos_label='spam')
prec = precision_score(y_train, pred_train, pos_label='spam')

print("Results of Naive Bayes on train data:","\nAccuracy:",acc, "\nRecall:",rec, "\nPrecision:",prec)

Train_Confusion Matrix:
 [[3378    0]
 [  71  450]] 

Results of Naive Bayes on train data: 
Accuracy: 0.9817902026160554 
Recall: 0.8637236084452975 
Precision: 1.0


##### Predict on test data

In [ ]:
pred = nb_clf.predict(X_test_tfidf) #predict on test data
prediction = confusion_matrix(y_test,pred) ##get confusion matrix for test predictions

In [ ]:
print("Test_Confusion matrix:\n", prediction,"\n")
acc = accuracy_score(y_test,pred)
rec = recall_score(y_test,pred,pos_label='spam')
prec=precision_score(y_test,pred,pos_label='spam')
print("Results of Naive Bayes on test data:","\nAcc: ",acc,"\nRec:", rec, "\nPrec:", prec)

Test_Confusion matrix:
 [[1445    1]
 [  56  170]] 

Results of Naive Bayes on test data: 
Acc:  0.9659090909090909 
Rec: 0.7522123893805309 
Prec: 0.9941520467836257


### Implementing Logistic Regression

In [ ]:
from sklearn import linear_model

logreg_train = linear_model.LogisticRegression()
lr_clf = logreg_train.fit(X_train_tfidf, y_train) #fit logreg model on train data

pred_lr_train=lr_clf.predict(X_train_tfidf) # predict on train data
pred_lr_test=lr_clf.predict(X_test_tfidf) # predict on test data


print("Train_Confusion matrix:\n", confusion_matrix(y_train,pred_lr_train))
print("Test_Confusion matrix:\n",confusion_matrix(y_test,pred_lr_test))

Train_Confusion matrix:
 [[3375    3]
 [ 129  392]]
Test_Confusion matrix:
 [[1443    3]
 [  72  154]]


In [ ]:
print("Results of Logistic regression on test data")
acc=accuracy_score(y_test,pred_lr_test)
rec=recall_score(y_test,pred_lr_test,pos_label='spam')
prec=precision_score(y_test,pred,pos_label='spam')
print("\nAcc: ",acc,"\nRec:", rec, "\nPrec:", prec)

Results of Logistic regression on test data

Acc:  0.9551435406698564 
Rec: 0.6814159292035398 
Prec: 0.9941520467836257
